# TrpCage Analysis

First we need to download the files from colab:


In [ ]:
import sys
sys.path.append("..")
import helpers
helpers.set_style()

In [ ]:
# copy code from colab
code =  'adz9IoTVAFHh'

!curl https://file.io/$key --output archive.zip

In [ ]:
# unzip

!unzip archive.zip

In [91]:

import numpy as np
import pytraj as pt
import nglview as nv

import matplotlib.pyplot as plt

from matplotlib.ticker import MaxNLocator
from scipy import stats


## Lets load the trajectory files

In [3]:
traj = pt.load('trp_cage_gb.dcd', top='trp_cage.prmtop' )

Let's visualize our trajectory to get a first idea what happend over the course of 20 ns to our linear peptide. 

In [92]:
view = nv.show_pytraj(traj)
view.clear_representations()
view.add_representation('licorice', selection='not hydrogen')
view.add_representation('cartoon')
view.add_representation('contact')
view.center()
view

NGLWidget(max_frame=199)

<div class="exercise admonition" name="6ex3" style="padding: 10px">
<p class="title">Exercise 3</p>
What type of structure is the folded Trp-cage miniprotein? List the main components contributing to this structure, including the residues which are responsible for their formation.
</div>

We will also load an experimental NMR structure from an experimental NMR ensemble.

```{note} Experimental structures Quick overview
:class: dropdown

Experimental structures can be obtained using three main methods:

### X-Ray diffraction 
Freeze the proteins and shoot x-rays at it. The diffraction pattern allows to reconstruct the electron density map in which we can fit a protein model.
The models are usually very accurate, may have crystal artefacts due to packing and low temperature and can be obtained for proteins from large to small. Some proteins are difficult to crystallize. 

### NMR 
In solution structure using nuclear magnetic resonance (usually carbon and hydrogen) using complicated pulse sequences. Allows to resolve dynamical properties of the protein but normally is limited to smaller proteins. 

### Cryo EM
The new kid on the block of structural biology. Works very well especially for large proteins, models are fit into a reconstructed map of the protein. Resolution tends to be lower than using X-Ray crystallography but sample preparation is much easier.
```

In [2]:
ref = pt.load('1l2y.pdb')

<div class="exercise admonition" name="6ex4" style="padding: 10px">
<p class="title">Exercise 4</p>
Monitor an individual hydrogen bond involved in a secondary structure, and provide the graph of the hydrogen bond length over time. Can you infer at which interval (in nanoseconds) the secondary structure forms?
</div>

## Align trajectory

Aligning a trajectory is an important step when analysing molecular dynamics simulations. We will choose a reference structure/frame and project each frame to the reference (e.g by minimizing RMSD to reference). 

In this case we align on the backbone nitrogen and carbon atoms. 

As a reference we use one structure from the experimental NMR ensemble (PDB `1l2y`).

In [13]:
aligned_traj = pt.align(traj, ref=ref, mask='@N,CA,C')

pytraj.Trajectory, 200 frames: 
Size: 0.001359 (GB)
<Topology: 304 atoms, 20 residues, 1 mols, non-PBC>
           

We can also compute the RMSD to the reference structure using a differnt function. 

In [61]:
rmsd_to_nmr = pt.rmsd(aligned_traj, ref=ref, mask='@N,CA,C')

Next, we plot this RMSD.

In [ ]:
fig, ax = plt.subplots(1)

times = np.linspace(0,20, num=traj.n_frames)

ax.set_title('RMSD to NMR structure')
ax.plot(times,rmsd_to_nmr)
ax.set_xlabel('time [ns]')
ax.set_ylabel('RMSD [$\AA$]')
plt.show()

an alternative is the RMSF

In [84]:
rmsf = pt.rmsf(aligned_traj[100:], mask="@CA")

In [85]:
resids = np.linspace(1, rmsf.shape[0], num=20, dtype=int)

In [ ]:
fig, ax = plt.subplots(1)

ax.set_title('RMSF')
ax.plot( resids, rmsf[:,1])
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_xlim(0,20)
ax.set_xlabel('residue')
ax.set_ylabel('RMSF [$\AA$]')
plt.show()

<div class="exercise admonition" name="6ex5" style="padding: 10px">
<p class="title">Exercise 5</p>
Explain the RMSD and RMSF plots.  Does the trajectory reach the same conformation as the experimental structure?
Which metric is more useful for the problem at hand? <b>Bonus:</b>  Provide a use case for the other metric. 
</div>

## Emergence of secondary structure

The central tryptophan in the Trp cage protein is located in the alpha helix. Here we track the dihedral angle along our simulation. When the protein forms ordered secondary structure elements we expect to see the dihedral only fluctuate a little. 

In [ ]:
# look at dihedral angle of Trp6 in the alpha helix
df_trp6_dihedrals = pt.multidihedral(aligned_traj, resrange='6', dtype='dataframe')
df_trp6_dihedrals.head(6)

In [ ]:
fig, ax = plt.subplots(1)

times = np.linspace(0,20, num=traj.n_frames)

binned_dihedral, bin_edges, binnumber = stats.binned_statistic(times, df_trp6_dihedrals['phi_6'], bins=30)

ax.set_title('Trp6 $\phi$ dihedral')
ax.plot(binned_dihedral)
ax.set_xlabel('time [ns]')
ax.set_ylabel('dihedral $[degree]$')
plt.show()

## Hbond analysis

We know that hydrogen bonds are very important for the formation of secondary structure elements. Let's count the number of hbonds per frame and bin them in 1 ns bins by taking the mean of 10 frames. 

In [28]:
hbond_data = pt.hbond(aligned_traj)

In [29]:
n_hbonds_per_frame = hbond_data.total_solute_hbonds()


In [54]:
# Bin every nanosecond (i.e. 10 snapshots)
times = np.linspace(0,20, num=traj.n_frames)
statistic, bin_edges, binnumber = stats.binned_statistic(times, n_hbonds_per_frame, bins=20)

In [ ]:
fig, ax = plt.subplots(1)

times = np.linspace(0,20, num=traj.n_frames)

ax.set_title('Average number of Hbonds')
ax.plot(statistic)
ax.set_xlabel('time [ns]')
ax.set_ylabel('n hbonds')
plt.show()

<div class="exercise admonition" name="6ex6" style="padding: 10px">
<p class="title">Exercise 6</p>
 Include the hbond graph in your report, and explain the observed trend with reference to the structural components of the Trp-cage miniprotein ?
</div>

## Visualize trajectory

Now let's compare the trajectory to one of the structures from the NMR structural ensemble to see how our linear Trp cage folded. 

In [17]:
view = nv.show_pytraj(aligned_traj)
view.clear_representations()
view.add_component('1l2y.pdb')
view.add_representation('licorice', selection='not hydrogen')
view.add_representation('cartoon')
view.add_representation('contact')

view.center()

view

NGLWidget(max_frame=199)

<div class="exercise admonition" name="6ex7" style="padding: 10px">
<p class="title">Exercise 7</p>
Why is it useful to constrain bond lengths for larger MD simulations (typically with the SHAKE algorithm)? Which bonds would you typically constrain in such a scenario, and why?
</div>

<div class="exercise admonition" name="6bex1" style="padding: 10px">
<p class="title">Bonus exercise 8</p>
Which properties do you need to take into account in order to select an appropriate timestep for your MD simulation? Are there any other reasons you might wish to reduce or increase this timestep?
</div>